In [1]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from graph_tool.all import *

import heapq
import requests
import json
import time
import csv
import itertools

In [2]:
def hex_to_int(id):
    '''Map patent id (string) to graph id (int).'''
    return int(id, 16)

def int_to_hex(id):
    '''Map graph id (int) to patent id (string).'''
    return '{0:0x}'.format(id)

In [3]:
DATA = './data/'
graph = load_graph(DATA + 'citations_graph_reversed.xml.gz')
graph

<Graph object, directed, with 8274991 vertices and 86284396 edges at 0x7fb7475ba2e8>

In [ ]:
def list_shortest_path(source, target, graph, verbose=False):
    '''Compute and print the shortest path between two patents.
    This function will pull the patent titles from the patentsview.org api.
    
    Keyword arguments:
    source -- Patent id of source node (string)
    target -- Patent id of target node (string)
    graph -- Patent graph
    verbose -- Set to true to print out shortest path (default=False)
    '''
    
    source_int_id = hex_to_int(source)
    target_int_id = hex_to_int(target)
    
    source_vertex = find_vertex(graph, graph.vertex_properties.id, source_int_id)
    if (len(source_vertex) == 0):
        print('Source not found in graph.')
        return None
    
    target_vertex = find_vertex(graph, graph.vertex_properties.id, target_int_id)
    if (len(target_vertex) == 0):
        print('Target not found in graph.')
        return None
    
    path = shortest_path(graph, source_vertex[0], target_vertex[0])
    
    if (len(path[0]) == 0):
        print('The graph contains no path from source to target.')
        return None
    
    if verbose:
        print('Shortest path:\n--------------')
        for idx, v in enumerate(path[0]):
            patent_id = int_to_hex(graph.vp.id[int(v)])
            patent_URL = 'http://www.patentsview.org/api/patents/query?q={"patent_number":"' + patent_id + '"}'
            patent_info = requests.get(patent_URL).json()
            if (patent_info['patents'] is None):
                patent_title = '-No title available-'
            else:
                patent_title = patent_info['patents'][0]['patent_title']
            print('ID: {}\t{}'.format(patent_id, patent_title))
            if (idx < len(path[0])-1):
                print('↓')
            
    return path

In [ ]:
path = list_shortest_path('2981877', '4309756', graph, verbose=True)

## Manual all shortest paths

In [4]:
class ShortestPathVisitor(BFSVisitor):
    def __init__(self, name, pred, dist):
        self.name = name
        self.pred = pred
        self.dist = dist
            
    def examine_edge(self, e):
        # Getting distance of next node
        next_dist = self.dist[e.source()] + 1
            
        # Checking if target has shorter connections
        if self.pred[e.target()]:
            if next_dist == self.dist[e.target()]:
                self.pred[e.target()].append(int(e.source()))
        else: # First to discover is shortest
            self.pred[e.target()].append(int(e.source()))
            self.dist[e.target()] = next_dist

In [5]:
def all_shortest_paths(graph, pred, source, sink):
    '''Returns all shortest paths from a source to a sink.
    Uses graph indices.
    Returns vertex set, edge set.'''
    if (source == sink):
        return set([sink]), set()
    
    vertices = set([sink])
    edges = set()
    
    for v in pred[sink]:
        edges.add((v,sink))
        v_rec, e_rec = all_shortest_paths(graph, pred, source, v)
        vertices = vertices | v_rec
        edges = edges | e_rec
        
    return vertices, edges

In [6]:
def all_shortest_paths_combined(graph, source, sink_list, dist, pred):
    vertices = set()
    edges = set()
    
    print('Computing all shortest paths from ' + source)    
    source_vertex = find_vertex(graph, graph.vertex_properties.id, hex_to_int(source))[0]
    bfs_search(graph, int(source_vertex), ShortestPathVisitor('SPV_' + source, pred, dist))
        
    for sink in sink_list:
        print('\tto ' + sink)
        sink_vertex = find_vertex(graph, graph.vertex_properties.id, hex_to_int(sink))[0]
        #if pred[sink]:
        if shortest_path(graph, source_vertex, sink_vertex)[0]:
            print('\t\tfound!')
            vs, es = all_shortest_paths(graph, pred, int(source_vertex), int(sink_vertex))
            vertices = vertices | vs
            edges = edges | es
        
    return vertices, edges      

Source Patents:

- ..'6285999' Google Page rank
- '9652896' Snapchat AR
- .'8825597' Dropbox
- '8225376' Facebook
- '7904382' Solar City
- '6955484' GoPro
- ..'8046721' Apple slide to unlock
- .'9063330' Oculus Rift
- 'D683268' Tesla car design
- '9134731' Autonomous driving
- ..'8930044' Autonomous medical drone
- '5194299' Post-it



Sink Patents:

- .'4136359' Apple II
- ..'2981877' Semiconductor / IC
- '416194' Electric motor - Nikola Tesla
- '174465' Telephone
- '1647' Telegraph
- '200521' Phonograph
- '223898' Light bulb
- X '430804' Electrical calculating system 
- ..'821393' Flying-machine
- '1867377' Sliced bread
- '1102653' Multistaged rocket
- '4022227' Method of concealing partial baldness 
- X '20050228218' Double anchor strapless dildo 
- '8540' Zipper
- '3009235' Velcro
- '1331952' Bouncy shoe
- .'1143542' Moving picture
- ..'676332' Apparatus for wireless telegraphy

In [7]:
targets = np.genfromtxt('highest_pr.csv', delimiter=',', usecols=(0), skip_header=True, dtype=str)

In [8]:
sink_list = targets

In [17]:
source = '676332'

In [18]:
dist = graph.new_vertex_property("int")
pred = graph.new_vertex_property("vector<int32_t>")

asp = all_shortest_paths_combined(graph, source, sink_list, dist, pred)

Computing all shortest paths from 676332
	to 4237224
	to 3813316
	to 4309756
	to 4558413
	to 4395486
	to 4683202
	to 4683195
	to 4298685
	to 3988545
	to 5523520
	to 3932805
	to 3747120
	to 4074232
	to 3956615
	to 3898390
	to 3749845
	to 4251824
	to 4812599
	to 4063220
	to 4358535
	to 3663762
	to 3798359
	to 5103459
	to 3906166
	to 3778614
	to 4032899
	to 4320500
	to 4200770
	to 4901307
	to 4539507
	to 4144412
	to 4463359
	to 4346442
	to 4723129
	to 3789832
	to 5572643
		found!
	to 4193131
	to 4315315
	to 4064521
	to 4203158
	to 4432057
	to 4405829
	to 4555775
	to 3890471
	to 4218582
	to 3766322
	to 3500142
	to 3840695
	to 4377849
	to 5056109
	to 4399531
	to 4196265
	to 3597549
	to 3798360
	to 4021726
	to 4464650
	to 3962539
	to 4591983
	to 3798605
	to 2921124
	to 4287592
	to 4243994
	to 3533086
	to 3906460
	to 4283771
	to 4343993
	to 4730340
	to 4367924
	to 4001783
	to 3657476
	to 4191860
	to 4194242
	to 4007450
	to 3852157
	to 4158236
	to 4769292
	to 4455619
	to 3878519
	to 4103315
	t

	to 4675285
	to 4438032
	to 4635208
	to 4214305
	to 4264808
	to 4503499
	to 4058430
	to 6029195
	to 4539653
	to 4516238
	to 3790663
	to 3959624
	to 4029963
	to 3916390
	to 4683194
	to 5835087
	to 4819159
	to 3940758
	to 4225919
	to 5744864
	to 4284852
	to 3773919
	to 4677552
	to 5623601
		found!
	to 4535060
	to 3810104
	to 6229506
	to 4272880
	to 4204227
	to 5163131
		found!
	to 3334629
	to 4740954
	to 4140126
	to 4536791
	to 3734594
	to 4330787
	to 4170782
	to 4186438
	to 3566358
	to 4016405
	to 3648125
	to 4497039
	to 3480914
	to 3879597
	to 3876208
	to 4642790
	to 4977455
	to 3851318
	to 4092732
	to 3755704
	to 4571456
	to 3715485
	to 3992158
	to 4494230
	to 5796952
	to 4386233
	to 5175867
	to 3977007
	to 4507751
	to 4672410
	to 5223409
	to 4827508
	to 3976995
	to 4580568
	to 5416011
	to 3965358
	to 3919711
	to 4669113
	to 3455625
	to 4566095
	to 4096571
	to 4944836
	to 4468732
	to 4553545
	to 3704345
	to 3757225
	to 3540431
	to 4366613
	to 3582787
	to 2556550
	to 4655771
	to 553442

In [19]:
max_dist = 0

for sink in sink_list:
    sink_vertex = find_vertex(graph, graph.vertex_properties.id, hex_to_int(sink))[0]
    sink_dist = dist[sink_vertex]
    if sink_dist > max_dist:
        max_dist = sink_dist

In [20]:
max_dist

14

## Convert paths to CSVs

In [21]:
def paths_vertices_to_csv(paths, filename, dist, max_dist):
    vertices = []
    
    for p in paths: # For each path
        for v in p[0]: # For all the vertices in that path
            patent_id = int_to_hex(graph.vp.id[int(v)])
            vertex_dist = '<[' + str(dist[v]) + ',' + str(max_dist) + ']>'
            z = dist[v]
            vertices.append([patent_id, vertex_dist, z])
            
    output_file = open(filename, 'w')
    with output_file:  
        writer = csv.writer(output_file)
        writer.writerow(['Id', 'Interval', '[z]'])
        writer.writerows(vertices)

    return vertices

In [22]:
vertices = paths_vertices_to_csv([asp], './path_vertices.csv', dist, max_dist)

In [23]:
def paths_edges_to_csv(paths, filename):
    edges = []
    
    for p in paths: # For each path
        for e in p[1]: # For all the edges in that path
            source_index = int(e[0])
            source_id = int_to_hex(graph.vp.id[source_index])
            target_index = int(e[1])
            target_id = int_to_hex(graph.vp.id[target_index])
            
            edges.append([target_id, source_id])
            
    output_file = open(filename, 'w')
    with output_file:  
        writer = csv.writer(output_file)
        writer.writerow(['Source', 'Target'])
        writer.writerows(edges)
        
    return edges

In [24]:
edges = paths_edges_to_csv([asp], './path_edges.csv')